## Experimental notebook for upscaling algorithm tests

In [ ]:
from PIL import Image
import os
import numpy as np
# Folder path containing the images
folder_path = "./data/downscaled_input/"

### upscaling using bicubic interpolation

In [ ]:
def upsample_image(image_path, output_size):
    # Open the image
    image = Image.open(image_path)

    # Perform downsampling using bicubic interpolation
    downscaled_image = image.resize(output_size, resample=Image.BICUBIC)

    return downscaled_image


# Defining folder for downscaled images serving for input for modelling (&upscaling)
output_folder_path = "./data/bicubic_upscaling_Marcus"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)


# Output size for upsampling (by a factor of 5)
output_size = (300, 440)

# Iterate over the files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image (optional)
    if filename.endswith((".jpg", ".jpeg", ".png")):
        # Construct the full path to the image file
        image_path = os.path.join(folder_path, filename)

        # Apply upsampling to the image
        upsampled_image = upsample_image(image_path, output_size)

        # Save the upscaled image
        output_filename = f"bicubic_up_{filename}"
        output_path = os.path.join(output_folder_path, output_filename)
        upsampled_image.save(output_path)


## CNN implementation

#### imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import os
from torch.utils.data import DataLoader, Dataset

#### old approach

In [ ]:
# input_data_dir = "./data/Input_images_models"
# desired_data_dir = "./data/original_images_models/"

# # transformation to tensors
# transform = transforms.Compose([
#     transforms.ToTensor(),  # Convert images to tensors
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize the image tensors
# ])

# # datasets for input images and desired images
# input_dataset = datasets.ImageFolder(input_data_dir, transform=transform)
# desired_dataset = datasets.ImageFolder(desired_data_dir, transform=transform)

# # data loaders for input images and desired images
# batch_size = 32
# shuffle = False
# num_workers = 4 
# input_loader = torch.utils.data.DataLoader(input_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
# desired_loader = torch.utils.data.DataLoader(desired_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)



In [ ]:
# # SRCNN model
# class SRCNN(nn.Module):
#     def __init__(self):
#         super(SRCNN, self).__init__()
#         self.interpolation = nn.Upsample(scale_factor=5, mode='bicubic')
#         self.conv1 = nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4)
#         self.relu1 = nn.ReLU()
#         self.conv2 = nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0)
#         self.relu2 = nn.ReLU()
#         self.conv3 = nn.Conv2d(32, 3, kernel_size=5, stride=1, padding=2)
#         self.relu3 = nn.ReLU()

#     def forward(self, x):
#         x = self.interpolation(x)
#         x = self.relu1(self.conv1(x))
#         x = self.relu2(self.conv2(x))
#         x = self.relu3(self.conv3(x))
#         return x

# # Training hardware
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # instance of the CNN model
# model = SRCNN().to(device)

# # hyperparameters
# learning_rate = 0.001
# num_epochs = 1

# # loss function and optimizer
# criterion = nn.MSELoss() # note: standard MSE is used, PSNR normally not used for training (just as metric at the end)
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # Training process
# for epoch in range(num_epochs):
#     for input_data, desired_data in zip(input_loader, desired_loader):
#         # Move input and desired images to device
#         input_images, _ = input_data
#         desired_images, _ = desired_data
#         input_images = input_images.to(device)
#         desired_images = desired_images.to(device)

#         # Forward pass
#         output_images = model(input_images)

#         # Calculate loss
#         loss = criterion(output_images, desired_images)

#         # Backward and optimize
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     # Print training loss per epoch
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


### data loading

In [3]:
low_res_folder = "./data/downscaled_input"
high_res_folder = "./data/data_final"

# dataset class
class CustomDataset(Dataset):
    def __init__(self, low_res_folder, high_res_folder, transform=None):
        self.low_res_images = sorted(os.listdir(low_res_folder))
        self.high_res_images = sorted(os.listdir(high_res_folder))
        self.transform = transform

    def __len__(self):
        return len(self.low_res_images)

    def __getitem__(self, index):
        low_res_image = Image.open(os.path.join(low_res_folder, self.low_res_images[index]))
        high_res_image = Image.open(os.path.join(high_res_folder, self.high_res_images[index]))

        if self.transform is not None:
            low_res_image = self.transform(low_res_image)
            high_res_image = self.transform(high_res_image)

        return low_res_image, high_res_image

# transform
transform = transforms.Compose([
    transforms.ToTensor()
])

# custom dataset
dataset = CustomDataset(low_res_folder, high_res_folder, transform=transform)

# Ssplitting into train and test sets
test_size = 0.2  # Proportion of data to be used for testing
dataset_size = len(dataset)
split = int(test_size * dataset_size)
train_indices = list(range(split))
test_indices = list(range(split, dataset_size))

# data loaders for train and test sets
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, sampler=torch.utils.data.SubsetRandomSampler(train_indices))
test_loader = DataLoader(dataset, batch_size=batch_size, sampler=torch.utils.data.SubsetRandomSampler(test_indices))

# number of samples in each set
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of test samples: {len(test_indices)}")


Number of training samples: 374
Number of test samples: 1496


### implementing SRCNN

In [ ]:
# SRCNN model
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=5, mode='bicubic')
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.interpolation(x)
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.relu3(self.conv3(x))
        return x

# Training hardware
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# instance of the CNN model
model = SRCNN().to(device)

# hyperparameters
learning_rate = 0.001
num_epochs = 1

# loss function and optimizer
criterion = nn.MSELoss() # note: standard MSE is used, PSNR normally not used for training (just as metric at the end)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training process
for epoch in range(num_epochs):
    for input_data, desired_data in train_loader:
        # Move input and desired images to device
        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print training loss per epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# saving the model
torch.save(model.state_dict(), "SRCNN.pth")